In [1]:
import tkinter as tk
from tkinter import messagebox, filedialog
import csv


# Employee Class
class Employee:
    def __init__(self, emp_id, name, position, salary, email):
        self.emp_id = emp_id
        self.name = name
        self.position = position
        self.salary = salary
        self.email = email


# Employee Manager Class
class EmployeeManager:
    def __init__(self, filename):
        self.filename = filename
        self.employees = self.load_data()

    def load_data(self):
        employees = {}
        try:
            with open(self.filename, mode="r") as file:
                reader = csv.DictReader(file)
                for row in reader:
                    emp = Employee(
                        emp_id=row['ID'],
                        name=row['Name'],
                        position=row['Position'],
                        salary=float(row['Salary']),
                        email=row['Email']
                    )
                    employees[emp.emp_id] = emp
        except FileNotFoundError:
            pass
        return employees

    def save_data(self):
        with open(self.filename, mode="w", newline="") as file:
            fieldnames = ['ID', 'Name', 'Position', 'Salary', 'Email']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            for emp in self.employees.values():
                writer.writerow({
                    'ID': emp.emp_id,
                    'Name': emp.name,
                    'Position': emp.position,
                    'Salary': emp.salary,
                    'Email': emp.email
                })

    def add_employee(self, emp):
        if emp.emp_id in self.employees:
            return False
        self.employees[emp.emp_id] = emp
        self.save_data()
        return True

    def update_employee(self, emp_id, **kwargs):
        if emp_id not in self.employees:
            return False
        emp = self.employees[emp_id]
        emp.name = kwargs.get('name', emp.name)
        emp.position = kwargs.get('position', emp.position)
        emp.salary = kwargs.get('salary', emp.salary)
        emp.email = kwargs.get('email', emp.email)
        self.save_data()
        return True

    def delete_employee(self, emp_id):
        if emp_id not in self.employees:
            return False
        del self.employees[emp_id]
        self.save_data()
        return True

    def search_employee(self, emp_id):
        return self.employees.get(emp_id, None)

    def export_data(self, export_filename):
        with open(export_filename, mode="w", newline="") as file:
            fieldnames = ['ID', 'Name', 'Position', 'Salary', 'Email']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            for emp in self.employees.values():
                writer.writerow({
                    'ID': emp.emp_id,
                    'Name': emp.name,
                    'Position': emp.position,
                    'Salary': emp.salary,
                    'Email': emp.email
                })


# GUI Application
class EmployeeApp:
    def __init__(self, root):
        self.root = root
        self.manager = EmployeeManager("employees.csv")
        self.emp_id_counter = len(self.manager.employees) + 1

        # Main Window
        self.root.title("Employee Management System")
        self.root.geometry("400x300")

        tk.Label(self.root, text="Welcome to Employee Management System", font=("Arial", 14)).pack(pady=20)
        tk.Button(self.root, text="Login as Employee", font=("Arial", 12), command=self.employee_login).pack(pady=10)
        tk.Button(self.root, text="Login as Manager", font=("Arial", 12), command=self.manager_login).pack(pady=10)

    def employee_login(self):
        # Employee Login Page
        self.employee_window = tk.Toplevel(self.root)
        self.employee_window.title("Employee Login")
        self.employee_window.geometry("400x300")

        tk.Label(self.employee_window, text="Are you a new or existing employee?", font=("Arial", 12)).pack(pady=10)

        tk.Button(self.employee_window, text="New Employee", command=self.new_employee).pack(pady=5)
        tk.Button(self.employee_window, text="Existing Employee", command=self.existing_employee).pack(pady=5)

    def new_employee(self):
        # New Employee Form
        self.new_emp_window = tk.Toplevel(self.employee_window)
        self.new_emp_window.title("New Employee Registration")
        self.new_emp_window.geometry("400x400")

        tk.Label(self.new_emp_window, text="Name").pack()
        self.name_entry = tk.Entry(self.new_emp_window)
        self.name_entry.pack()

        tk.Label(self.new_emp_window, text="Position").pack()
        self.position_entry = tk.Entry(self.new_emp_window)
        self.position_entry.pack()

        tk.Label(self.new_emp_window, text="Salary").pack()
        self.salary_entry = tk.Entry(self.new_emp_window)
        self.salary_entry.pack()

        tk.Label(self.new_emp_window, text="Email").pack()
        self.email_entry = tk.Entry(self.new_emp_window)
        self.email_entry.pack()

        tk.Button(self.new_emp_window, text="Register", command=self.register_new_employee).pack(pady=20)

    def register_new_employee(self):
        # Registering New Employee
        name = self.name_entry.get().strip()
        position = self.position_entry.get().strip()
        salary = self.salary_entry.get()
        salary = self.salary_entry.get().strip()
        email = self.email_entry.get().strip()

        # Generate unique employee ID
        emp_id = f"E{self.emp_id_counter}"

        # Validate input
        if not name or not position or not salary or not email:
            messagebox.showerror("Error", "All fields are required.")
            return

        try:
            salary = float(salary)  # Ensure salary is numeric
        except ValueError:
            messagebox.showerror("Error", "Salary must be a numeric value.")
            return

        # Add employee to the system
        emp = Employee(emp_id=emp_id, name=name, position=position, salary=salary, email=email)
        if self.manager.add_employee(emp):
            messagebox.showinfo("Success", f"Employee {name} registered successfully with ID: {emp_id}!")
            self.emp_id_counter += 1
            self.new_emp_window.destroy()  # Close the registration form
        else:
            messagebox.showerror("Error", "An error occurred while registering the employee.")

    def existing_employee(self):
        # Existing Employee Lookup
        self.existing_emp_window = tk.Toplevel(self.employee_window)
        self.existing_emp_window.title("Existing Employee Lookup")
        self.existing_emp_window.geometry("400x200")

        tk.Label(self.existing_emp_window, text="Enter Employee ID", font=("Arial", 12)).pack(pady=10)
        self.emp_id_entry = tk.Entry(self.existing_emp_window)
        self.emp_id_entry.pack()

        tk.Button(self.existing_emp_window, text="Search", command=self.lookup_employee).pack(pady=10)

    def lookup_employee(self):
        emp_id = self.emp_id_entry.get().strip()
        emp = self.manager.search_employee(emp_id)

        if emp:
            messagebox.showinfo(
                "Employee Details",
                f"ID: {emp.emp_id}\nName: {emp.name}\nPosition: {emp.position}\nSalary: {emp.salary}\nEmail: {emp.email}"
            )
        else:
            messagebox.showerror("Error", "Employee ID not found. Please try again.")

    def manager_login(self):
        # Manager Login Page
        self.manager_window = tk.Toplevel(self.root)
        self.manager_window.title("Manager Panel")
        self.manager_window.geometry("400x400")

        tk.Label(self.manager_window, text="Manager Options", font=("Arial", 14)).pack(pady=20)

        tk.Button(self.manager_window, text="Update Employee Data", command=self.update_employee_data).pack(pady=10)
        tk.Button(self.manager_window, text="Delete Employee", command=self.delete_employee).pack(pady=10)
        tk.Button(self.manager_window, text="Export All Data to CSV", command=self.export_data).pack(pady=10)
        tk.Button(self.manager_window, text="View All Employees", command=self.list_all_employees).pack(pady=10)

    def update_employee_data(self):
        # Update Employee Data
        self.update_emp_window = tk.Toplevel(self.manager_window)
        self.update_emp_window.title("Update Employee Data")
        self.update_emp_window.geometry("400x400")

        tk.Label(self.update_emp_window, text="Enter Employee ID to Update", font=("Arial", 12)).pack(pady=10)
        self.update_emp_id_entry = tk.Entry(self.update_emp_window)
        self.update_emp_id_entry.pack()

        tk.Button(self.update_emp_window, text="Search", command=self.edit_employee_data).pack(pady=10)

    def edit_employee_data(self):
        emp_id = self.update_emp_id_entry.get().strip()
        emp = self.manager.search_employee(emp_id)

        if emp:
            tk.Label(self.update_emp_window, text="Update Details").pack(pady=10)

            tk.Label(self.update_emp_window, text="Name").pack()
            self.new_name_entry = tk.Entry(self.update_emp_window)
            self.new_name_entry.insert(0, emp.name)
            self.new_name_entry.pack()

            tk.Label(self.update_emp_window, text="Position").pack()
            self.new_position_entry = tk.Entry(self.update_emp_window)
            self.new_position_entry.insert(0, emp.position)
            self.new_position_entry.pack()

            tk.Label(self.update_emp_window, text="Salary").pack()
            self.new_salary_entry = tk.Entry(self.update_emp_window)
            self.new_salary_entry.insert(0, emp.salary)
            self.new_salary_entry.pack()

            tk.Label(self.update_emp_window, text="Email").pack()
            self.new_email_entry = tk.Entry(self.update_emp_window)
            self.new_email_entry.insert(0, emp.email)
            self.new_email_entry.pack()

            tk.Button(self.update_emp_window, text="Update", command=lambda: self.save_updated_employee(emp_id)).pack(pady=10)
        else:
            messagebox.showerror("Error", "Employee ID not found.")

    def save_updated_employee(self, emp_id):
        name = self.new_name_entry.get().strip()
        position = self.new_position_entry.get().strip()
        salary = self.new_salary_entry.get().strip()
        email = self.new_email_entry.get().strip()

        try:
            salary = float(salary)
        except ValueError:
            messagebox.showerror("Error", "Salary must be a numeric value.")
            return

        if self.manager.update_employee(emp_id, name=name, position=position, salary=salary, email=email):
            messagebox.showinfo("Success", "Employee data updated successfully!")
            self.update_emp_window.destroy()
        else:
            messagebox.showerror("Error", "Failed to update employee data.")

    def delete_employee(self):
        # Delete Employee
        self.delete_emp_window = tk.Toplevel(self.manager_window)
        self.delete_emp_window.title("Delete Employee")
        self.delete_emp_window.geometry("400x200")

        tk.Label(self.delete_emp_window, text="Enter Employee ID to Delete", font=("Arial", 12)).pack(pady=10)
        self.delete_emp_id_entry = tk.Entry(self.delete_emp_window)
        self.delete_emp_id_entry.pack()

        tk.Button(self.delete_emp_window, text="Delete", command=self.remove_employee).pack(pady=10)

    def remove_employee(self):
        emp_id = self.delete_emp_id_entry.get().strip()

        if self.manager.delete_employee(emp_id):
            messagebox.showinfo("Success", "Employee deleted successfully!")
            self.delete_emp_window.destroy()
        else:
            messagebox.showerror("Error", "Employee ID not found.")

    def export_data(self):
        # Export Data to CSV
        file_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])
        if file_path:
            self.manager.export_data(file_path)
            messagebox.showinfo("Success", "Data exported successfully!")

    def list_all_employees(self):
        # List All Employees
        employees = "\n".join(
            [f"ID: {emp.emp_id}, Name: {emp.name}, Position: {emp.position}, Salary: {emp.salary}, Email: {emp.email}"
             for emp in self.manager.employees.values()]
        )
        messagebox.showinfo("All Employees", employees if employees else "No employees found.")


# Main Application
if __name__ == "__main__":
    root = tk.Tk()
    app = EmployeeApp(root)
    root.mainloop()
